In [ ]:
import arcticdb as adb
import json
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import dtale
from sklearn.decomposition import PCA
from tradingo.utils import get_instruments, get_config, with_instrument_details

In [ ]:
a = adb.Arctic("lmdb:///home/rory/dev/airflow/test/arctic.db")
signals = a.get_library("MODEL_SIGNALS")
backtests = a.get_library("BACKTEST")
prices = a.get_library("ASSET_PRICES")
portfolios = a.get_library("PORTFOLIO")

In [ ]:
instruments = get_instruments(get_config())

orig_prices = prices.read("ETFT.ADJ_CLOSE.yfinance")
start_dates = orig_prices.data.apply(lambda i: i.first_valid_index()).rename("start_date")
instruments = instruments.merge(start_dates, left_index=True, right_index=True, how='left')

In [ ]:
PRICES = with_instrument_details((orig_prices.data[
                instruments[(instruments.start_date < pd.Timestamp("2018-03-01"))].index
          ]).dropna(), instruments, columns=["Asset Class"])
RETURNS = np.log(PRICES/PRICES.shift()).dropna()

In [ ]:
ORIG_PRICES = prices.read("ETFT.CLOSE.yfinance").data

In [ ]:
PRICES.transpose().groupby(["Asset Class"]).mean().transpose().plot()

In [ ]:
(1+RETURNS.loc[:,"Commodities & Digital Assets"].transpose().merge(instruments.Name, left_index=True, right_index=True).reset_index(drop=True).set_index(["Name"]).transpose()).cumprod().plot()

In [ ]:
pca = PCA()
pca.fit(RETURNS)
pd.DataFrame(pca.components_, index=RETURNS.columns)
df = pd.DataFrame(pca.components_, columns=RETURNS.columns)

In [ ]:
signals.list_symbols()

In [ ]:
signals.read("ETFT.trend.ewmac_128_64").data["AIA"].plot()

In [ ]:
raw_prices = prices.read('ETFT.CLOSE.yfinance').data

In [ ]:
(trend_positions.div(trend_positions.transpose().sum(), axis=0) * 70_000)/raw_prices

In [ ]:
(np.sqrt(260) * signals.read("ETFT.trend.vol_64").data["USRT"]).plot()

In [ ]:
instruments[instruments["Asset Class"].str.contains("Commodit")]

In [ ]:
pf = portfolios.read("ETFT.trend.RAW.SHARES").data

In [ ]:
trades = pf.round().diff()

position

In [ ]:
trades = pf.fillna(0).round() - pf.fillna(0).round().shift()
position = trades.cumsum()

In [ ]:
(position * ORIG_PRICES).sum(axis=1) - (trades * ORIG_PRICES).sum(axis=1).cumsum().rename("account").plot()

In [ ]:
(trades  *ORIG_PRICES).supricesm

In [ ]:
pf